<a href="https://colab.research.google.com/github/frapasti/Human-Hands/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# #TODOs
* Find the best architecture on a small dataset (See 3. for details)
* Create a batch generation function in order to not fill the memory of Colab --> DONE! ;)
* Train the best architecture on the new batch generator







# IMPORTS

* Import all the needed libraries for the notebook
* Mount the drive containing the datasets
* Unzip the a small part of the dataset







In [ ]:
!pip install natsort &> /dev/null
import os
from natsort import natsorted
import cv2                   

def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

from google.colab import drive
drive.mount('/content/drive')

!unzip drive/My\ Drive/Datasets/egoHands.zip &> /dev/null #no need to fill the output stream with all the inflations

Mounted at /content/drive


In [ ]:
!unzip drive/My\ Drive/Datasets/ColorDA.zip &> /dev/null

In [ ]:
!unzip drive/My\ Drive/Datasets/handsOverFaces.zip &> /dev/null

In [ ]:
!unzip drive/My\ Drive/Datasets/noHands.zip &> /dev/null

# Data preparation

Get all the bboxes path and all the img_paths
Additionally read all the text files corresponding to the bounding boxes and append them to a list.
**bboxes** contains lists of bounding boxes, one per image

In [ ]:
#Get from the unzipped dataset the list of all the paths of images and txt files
#listdir returns unsorted so use natsorted method!

#paths needed for the training set
bboxes_paths = natsorted(listdir_fullpath('/content/egoHands/boundingboxes'))
img_paths = natsorted(listdir_fullpath('/content/egoHands/frames'))

assert len(bboxes_paths) == len(img_paths), f"Number of bboxes_paths doesn't correspond with number of images!"

#Read all the text files and create a list of list of bounding boxes for the training, one list per image 
bboxes = []*len(bboxes_paths)
for path in bboxes_paths:
  bboxes_it = []
  file = open(path, 'r')
  Lines = file.readlines()
  for line in Lines:
    box = line.split(',')
    if(not(int(box[0]) == 0 and int(box[1]) == 0 and int(box[2]) == 0 and int(box[3]) == 0)):
      bboxes_it.append([int(x) for x in box])
  bboxes.append(bboxes_it)

assert(len(bboxes) == len(img_paths))

In [ ]:
colorBboxes_paths = natsorted(listdir_fullpath('/content/ColorDA/boundingboxes'))
colorImg_paths = natsorted(listdir_fullpath('/content/ColorDA/frames'))

assert len(colorBboxes_paths) == len(colorImg_paths), f"Number of bboxes_paths doesn't correspond with number of images!"

for path in colorBboxes_paths:
  bboxes_it = []
  file = open(path, 'r')
  Lines = file.readlines()
  for line in Lines:
    bboxes_it.append([int(float(x)) for x in line.split(',')])
  bboxes.append(bboxes_it)

for path in colorImg_paths:
  img_paths.append(path)

assert(len(bboxes) == len(img_paths))

In [ ]:
noHands_paths = natsorted(listdir_fullpath('/content/noHands'))

for path in noHands_paths:
  img_paths.append(path)
  bboxes.append([])

assert(len(bboxes) == len(img_paths))

In [ ]:
hofBboxes_paths = natsorted(listdir_fullpath('/content/handsOverFaces/boundingboxes'))
hofImg_paths = natsorted(listdir_fullpath('/content/handsOverFaces/frames'))

assert len(hofBboxes_paths) == len(hofImg_paths), f"Number of bboxes_paths doesn't correspond with number of images!"

for path in hofBboxes_paths:
  bboxes_it = []
  file = open(path, 'r')
  Lines = file.readlines()
  for line in Lines:
    bboxes_it.append([int(float(x)) for x in line.split(',')])
  bboxes.append(bboxes_it)

for path in hofImg_paths:
  img_paths.append(path)

assert(len(bboxes) == len(img_paths))

In [ ]:
import cv2 
import os 
for i in range(0, len(img_paths)):
  path = img_paths[i]
  directory = r'/content/drive/MyDrive/Datasets/data/obj'
  img = cv2.imread(path) 
  os.chdir(directory) 
  
  filename = "/content/drive/MyDrive/Datasets/data/obj/" + str(i)+".jpg"
  cv2.imwrite(filename, img) 


In [ ]:
def cocoToYolo(img_path, bbox):
  img = cv2.imread(img_path)
  img_height = img.shape[0]
  img_width = img.shape[1]

  x = bbox[0]
  y = bbox[1]
  w = bbox[2]
  h = bbox[3]

  dw = 1/img_width
  dh = 1/img_height

  x_center = x + w/2.0
  y_center = y + h/2.0

  x_new = x_center*dw
  y_new = y_center*dh
  w_new = w*dw
  h_new = h*dh

  return [0, x_new, y_new, w_new, h_new]

In [ ]:
import numpy as np
for i in range (0, len(bboxes)):
  if(bboxes[i]==[]):
    with open('/content/drive/MyDrive/Datasets/data/obj/'+str(i)+'.txt', 'w') as fp:
      pass
  else:
    yolo_bbox = []
    for box in bboxes[i]:
      yolo_bbox.append(cocoToYolo(img_paths[i], box))
    mat = np.matrix(yolo_bbox)
    with open('/content/drive/MyDrive/Datasets/data/obj/'+str(i)+'.txt','wb') as f:
      for line in mat:
          np.savetxt(f, line, fmt="%.0i " "%.10f " "%.10f " "%.10f " "%.10f")